# Step 2. Controlled generation of novel molecules using MolMIM

This notebook is adapted from the [custom oracles notebook](https://github.com/NVIDIA/BioNeMo/blob/main/examples/service/notebooks/cma_custom_oracles.ipynb) from NVIDIA bionemo python client repository.

This tutorial demonstrates how to use the molmim_cma package to optimize exploration of the [MolMIM](https://arxiv.org/abs/2208.09016) model's latent space to generate molecules
with properties of interest. To do this, we use the [CMA-ES](https://en.wikipedia.org/wiki/CMA-ES) genetic optimization algorithm. The basic steps of optimization are:

1. Decode latent representations into smiles strings
2. Score generated smiles strings based on the desired oracle function
3. Update the CMA-ES algorithm with the smiles/score pairing
4. Ask the CMA-ES algorithm for a new set of latent space representations to sample.

In [ ]:
!pip install rdkit python-dotenv pandas numpy matplotlib loguru py3dmol

In [ ]:
# load environment variables from .env file, which contains API key
import pickle
from typing import List
import numpy as np

from rdkit.Chem.QED import qed as rdkit_qed
from rdkit import Chem

import matplotlib.pyplot as plt
from IPython.display import display, clear_output

from rdkit.Chem.QED import qed
from rdkit.Chem import AllChem
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
from rdkit.DataStructs import TanimotoSimilarity
import ast
from google.colab import userdata
import os, shutil, requests
import pandas as pd

Then we can import and install library dependencies.

Now, let's set  up our molecule, which the PDB ID SZD (a malic enzyme inhibitor)

We'll run three replicates of optimization simulataneously - you could also seed different molecules here.

## 2.3 Set up the starting molecule

In [ ]:
smis = "O=C(Nc1cccc(S(=O)(=O)Nc2ccc(OC(F)(F)F)cc2)c1)c1cc(F)cc(F)c1"

mol = Chem.MolFromSmiles(smis)
qed_score = rdkit_qed(mol)
print(f"Original QED: {qed_score}")
mol

## Guided Molecular Generation with CMA-ES

In contrast to the random sampling of the latent space described (which is available when you self-host MolMIM NIM, see [doc](https://docs.nvidia.com/nim/bionemo/molmim/latest/endpoints.html#)), we can use a black box optimizer, called CMA-ES, to perform guided optimization of the a molecule's property through sampling of the latent space. In the blocks below, we use CMA-ES to optimize the QED score of the generated molecules while preseving a level of similary to the seed molecule, PDB ID SZD.

This first block initializes variables to store the results of the script and sets up the parameters for the experiment. It defines three lists to store the counts of valid SMILES strings, average Tanimoto similarities, and average QED scores. It also creates an empty dictionary to store the results and a list of minimum similarities to be used in the experiment, ranging from 0.1 to 0.7 with 3 evenly spaced values.

In [ ]:
# Create a dictionary to store the results
results = {}

# Create a list of minimum similarities
num_min_sims = 3
min_sims = np.linspace(0.1, 0.7, num_min_sims)

The following block is the main loop of the script, where it iterates over each minimum similarity value in the `min_sims` list. For each minimum similarity, it generates molecules using the CMA-ES algorithm, calculates the Tanimoto similarity and QED score for each SMILES string, and stores the results. The results are stored in a dictionary called `results`, where the keys are the minimum similarity values and the values are dictionaries containing the number of valid SMILES strings, average Tanimoto similarity, and average QED score. After the loop, the results are converted into a Pandas dataframe for further analysis.

In [ ]:
def tanimoto_similarity(smiles, reference: str):
    # Get fingerprint params
    fingerprint_radius_param = 2
    fingerprint_nbits = 2048

    # Handle the reference molecule
    reference_mol = Chem.MolFromSmiles(reference)
    reference_fingerprint = GetMorganFingerprintAsBitVect(
        reference_mol, radius=fingerprint_radius_param, nBits=fingerprint_nbits
    )

    # Validate the other molecule
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return 0

    fingerprint = GetMorganFingerprintAsBitVect(mol, radius=fingerprint_radius_param, nBits=fingerprint_nbits)

    # Calculate and return the Tanimoto similarity
    return TanimotoSimilarity(fingerprint, reference_fingerprint)

In [ ]:
# import ast
# from google.colab import userdata

API_KEY = userdata.get('API_KEY')
# print(API_KEY)

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Accept": "application/json",
}

invoke_url = "https://health.api.nvidia.com/v1/biology/nvidia/molmim/generate"

session = requests.Session()

# Loop through each minimum similarity value
for min_sim in min_sims:
    # Create a dictionary to store the results for this min_sim
    min_sim_results = {'smiles': [], 'num_smiles': [], 'tanimoto_similarity': [], 'qed_score': []}

    # Create the request payload
    payload = {
      "smi": smis,
      "algorithm": "CMA-ES",
      "num_molecules": 10,
      "property_name": "QED",
      "minimize": False,
      "min_similarity": min_sim,
      "particles": 20,
      "iterations": 2,
    }

    # Send the request and get the response
    response = session.post(invoke_url, headers=headers, json=payload)
    response.raise_for_status()
    response_json = response.json()
    print(f"*************** min_sim: {min_sim} ********************")
    print(f"response_json: \n"
          f"{response_json}")

    # Extract the generated SMILES
    gen_smiles_list = [i['sample'] for i in ast.literal_eval(response_json['molecules'])]
    print(f"gen_smiles_list: \n"
          f"{gen_smiles_list}")
    # get the molecule objects out of valid SMILES
    valid_mol_list = [mol for smiles in gen_smiles_list if (mol := Chem.MolFromSmiles(smiles))]
    # convert to canonical SMILES & deduplicate
    canonical_smiles = set()
    for mol in valid_mol_list:
        canonical_smi = Chem.MolToSmiles(mol, canonical=True)
        canonical_smiles.add(canonical_smi)
    canonical_smiles_list = list(canonical_smiles)
    print(f"canonical_smiles_list: \n"
          f"{canonical_smiles_list}")

    # Calculate Tanimoto similarity and QED score for each valid SMILES
    for smiles in canonical_smiles_list:
        tanimoto = tanimoto_similarity(smiles, smis)
        mol = Chem.MolFromSmiles(smiles)
        qed_score = qed(mol)
        min_sim_results['tanimoto_similarity'].append(tanimoto)
        min_sim_results['qed_score'].append(qed_score)

    # Update min_sim_results - get the average of Tanimoto and QED scores, store generated SMILES
    min_sim_results['tanimoto_similarity'] = np.mean(min_sim_results['tanimoto_similarity'])
    min_sim_results['qed_score'] = np.mean(min_sim_results['qed_score'])
    min_sim_results['num_smiles'] = len(canonical_smiles_list)
    min_sim_results['smiles'] = canonical_smiles_list

    # Store the results for this min_sim
    results[min_sim] = min_sim_results

In [ ]:
# Create a Pandas dataframe from the results
# df = pd.DataFrame(results).T.reset_index()
# Specify the keys you want to include
keys_to_include = ['num_smiles', 'tanimoto_similarity', 'qed_score']
# Create the DataFrame, selecting only the specified keys from each inner dictionary
df = pd.DataFrame({k: {key: val[key] for key in keys_to_include} for k, val in results.items()}).T.reset_index()
print(df)

The following block creates three plots to visualize the results of the script. The first plot shows the number of valid SMILES strings generated at each minimum similarity threshold. The second plot shows the average Tanimoto similarity of the generated molecules at each minimum similarity threshold. The third plot shows the average QED score of the generated molecules at each minimum similarity threshold. The plots are arranged in a single figure with three subplots, and the figure is displayed using the `plt.show()` function.

In [ ]:
# Create the plots
plt.figure(figsize=(20, 4))

# Plot the number of valid SMILES strings at each min_sim
plt.subplot(1, 3, 1)
plt.plot(df['index'], df['num_smiles'], linestyle='-', marker='o')
plt.xlabel('Minimum similarity')
plt.ylabel('Number of valid SMILES strings')
plt.title('Number of valid SMILES strings at each minimum similarity')

# Plot the average Tanimoto similarity at each radius
plt.subplot(1, 3, 2)
plt.plot(df['index'], df['tanimoto_similarity'], linestyle='-', marker='o')
plt.xlabel('Minimum similarity')
plt.ylabel('Average Tanimoto similarity')
plt.title('Average Tanimoto similarity at each minimum similarity')

# Plot the average QED score at each radius
plt.subplot(1, 3, 3)
plt.plot(df['index'], df['qed_score'], linestyle='-', marker='o')
plt.xlabel('Minimum similarity')
plt.ylabel('Average QED score')
plt.title('Average QED score at each minimum similarity')

plt.tight_layout()
plt.show()

<!-- ## 2.7 [Optional] Save the generated molecules -->

## 2.8 Preprocess the generated molecules for diffdock

In [ ]:
# choose the first N molecules from the individual runs to use for docking,
# for demo purpose we will set it to 3 for now.
N_molecules_for_docking = 3

# for each sublists, choose the first N molecules
molecules = [
    molecule
    for min_sim_key, min_sim_val in results.items()
    for molecule in min_sim_val['smiles'][:N_molecules_for_docking]
]
print(molecules)

In [ ]:
def prepare_output_directory(output):
    """
    Prepare the output directory
    output: str, the output directory
    return: None
    """
    # overwrite the output directory
    if os.path.exists(output):
        shutil.rmtree(output)
    os.makedirs(output)

In [ ]:
# converts SMILES to SDF (generate 1 conformer for each smile)
# overwrite output dir
output_dir = "/content/output/molmim_result"
output_dir_clean = os.path.join(output_dir, "clean_mols")
prepare_output_directory(output_dir_clean)

# convert to SDF
output_sdf_files = []
for i, smiles in enumerate(molecules):
    mol = Chem.MolFromSmiles(smiles)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol)
    AllChem.UFFOptimizeMolecule(mol)

    # save the clean file
    path = f"{output_dir_clean}/molecule_{i}.sdf"
    w = Chem.SDWriter(path)
    w.write(mol)
    w.close()
    print(f"Converted SMILES to SDF: {smiles}")
    output_sdf_files.append(path)

# here are the paths to the generated molecules in SDF format, which can be used for docking
print(output_sdf_files)

# **Download SDF files**

In [ ]:
output_dir_clean

In [ ]:
# Step 1: Zip the directory
zip_filename = "clean_mols.zip"
!cd {output_dir_clean} && zip -r {zip_filename} .

In [ ]:
# Step 2: Download the zipped file
from google.colab import files
files.download(os.path.join(output_dir_clean, zip_filename))